# 计算香农熵
熵定义为信息的期望值（表示形式与数据期望相同，也可以看成一种数据期望），如果待分类的事物可能划分在多个分类之中，则符号$x_{i}定义为:$
$$l(x_{i}) = -log_{2}p(x_{i})$$其中$p(x_{i})$是选择分类的概率。为了计算熵，需要计算所有类别所有可能值包含的信息期望值，通过下面的公式得到：
$$H = -\sum ^n_{i=1}p(x_{i})log_{2}p(x_{i})$$

>　发现一个latex技巧，1个美刀符号之间的$latex$公式不会换行, 两个美刀符号之间的公式会在前后换行。

# 计算信息不存度(impurity)
不存度是衡量信息

In [ ]:
# 具体算法
from math import log

def calcShannonEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts = dict()
    for fectVec in dataSet:
        currentLabel = fectVec[-1]
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key]) / numEntries
        shannonEnt -= prob * log(prob, 2)
    return shannonEnt


In [ ]:
def createDataSet():
    dataSet = [[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
    labels = ['no surfacing', 'flippers']
    return dataSet, labels


In [30]:
myDat, labels = createDataSet()
calcShannonEnt(myDat)

0.9709505944546686

In [31]:
myDat[0][-1] = 'maybe'
calcShannonEnt(myDat)

1.3709505944546687

# 划分数据集
分类算法除了需要测量信息熵，还需要划分数据集，度量划分数据集的熵，以便判断当前是否正确划分了数据集。
我们将对每个特征划分数据集的结果计算一次信息熵，然后判断按照哪个特征划分数据集是最好的划分方式。

In [20]:
def splitDataSet(dataSet, axis, value):  #　带划分的数据集、划分数据集的特征、需要返回的特征的值
    retDataSet = list()
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

# 选择最好的划分方式

In [9]:
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0
    bestFeature = -1
    for i in range(numFeatures):
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet) / float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)
        infoGain = baseEntropy - newEntropy
        if(infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature

In [ ]:
myDat, labels = createDataSet()
print(chooseBestFeatureToSplit(myDat))

0


In [ ]:
myDat

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [12]:
import operator

def majorityCnt(classList):
    classCount = dict()
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]


# 创建树原理
算法的原理就是尽量减少多个特征带来的影响，首先找到最优特征，该特征下有多种属性，然后通过不断地划分数据集找到满足该属性的数据集，计算数据集中标签出现最多的种类就是该属性对应的种类。

In [23]:
# 创建树
def createTree(dataSet, labels):
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    if len(dataSet[0]) == 1:
        return majorityCnt(classList)
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel: {}}
    del (labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value), subLabels)
    return myTree


In [ ]:
myDat, labels = createDataSet()
print(myDat, labels)
print(createTree(myDat, labels))

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']] ['no surfacing', 'flippers']
{'no surfacing': {'flippers': 'yes'}}


In [1]:
import matplotlib.pyplot as plt

In [ ]:
decisionNode = dict(boxstyle='sawtooth', fc='0.8')
leafNode = dict(boxstyle='roun4', fc='0.8')
